In [1]:
import time
import json
import numpy as np
import matplotlib.pyplot as plt

from pc import PS
from modules import ADC,DAC,CHIP
from cimCommand import CMD,CmdData,Packet
from cimCommand.singleCmdInfo import *

from util import plot_v_cond

In [2]:
chip=CHIP(PS(host="192.168.1.10", port = 7, debug=0),init=True)
chip.set_device_cfg(deviceType=0)

Connected to 192.168.1.10:7
local ip: 192.168.1.15 local port: 59912


# 1. 读操作

## 1.1 第一种读方式
- 需要自己设置读写模式
- 需要设置tia的增益
- 没有针对行列bank切换时的优化
- check_tia=True表示会检查16路tia(16路并行读), False表示不会检查
- sum=True表示是否可以多行相加, False表示每行单独读
- **不建议使用**(单点读会更快)

In [ ]:
chip.set_op_mode2(read=True,from_row=True)
chip.set_tia_gain(1)
voltage = chip.read2(row_index=[i for i in range(256)],col_index=[i for i in range(256)],read_voltage=0.1,tg=5,check_tia=True,sum=False)

## 1.2 第二种读方式
- 有针对行列bank切换时的优化
- 16路并行读
- 但是只能读任意行选和列选交叉的块
- out_type控制返回值, 0返回电压, 1返回电导(uS)，2返回电阻(kΩ)
- 没有求和功能

In [ ]:
voltage = chip.read_crossbar2(row_index=[i for i in range(256)],col_index=[i for i in range(256)]
                              ,read_voltage=0.1,tg=5,gain=1,from_row=True,out_type=0)
cond = chip.voltage_to_cond(voltage=voltage)
resistance = chip.voltage_to_resistance(voltage=voltage)

## 1.3 第三种读方式
- 逐点读,crossbar中值大于0即是需要读的点
- 针对行列bank切换进行优化过
- 相对16路并行读，会慢一点, 但是相对会更准确

In [ ]:
voltage = chip.read_point2(crossbar=np.ones((256,256)),read_voltage=0.1,tg=5,gain=1,from_row=True,out_type=0)
cond = chip.voltage_to_cond(voltage=voltage)
resistance = chip.voltage_to_resistance(voltage=voltage)

# 2. 写操作

## 2.1 第一种写方式
- 需要自己设置读写模式
- 需要自己设置脉冲宽度
- 行列交叉的块写
- 没有对行列选通的bank进行优化

In [ ]:
# set操作是从行给电压, 故from_row=True
chip.set_op_mode2(read=False,from_row=True)
chip.set_pulse_width(50e-6)
chip.write2(row_index=[i for i in range(256)],col_index=[i for i in range(256)],write_voltage=0,tg=0)

# reset操作是从列给电压, 故from_row=False
chip.set_op_mode2(read=False,from_row=False)
chip.set_pulse_width(50e-6)
chip.write2(row_index=[i for i in range(256)],col_index=[i for i in range(256)],write_voltage=0,tg=0)

## 2.2 第二种写方式(块)
- 有对行列bank的切换进行优化过
- 只能写行列交叉的块
-  set_device=True表示对选中的器件set, False表示对选中的器件进行reset

In [ ]:
chip.write_crossbar2(row_index=[i for i in range(256)],col_index=[i for i in range(256)],write_voltage=0,tg=0,pulse_width=50e-6,set_device=True)

# 2.3 第三种写方式(逐点)
- 针对行列bank切换优化过
- crossbar中>0表示对该器件写
-  set_device=True表示对选中的器件set, False表示对选中的器件进行reset

In [ ]:
condition_reset = np.ones((256,256))
condition_set = np.ones((256,256))
chip.write_point2(crossbar=condition_reset,write_voltage=2,tg=5,pulse_width=1000e-6,set_device=False)
chip.write_point2(crossbar=condition_set,write_voltage=3,tg=2.5,pulse_width=1000e-6,set_device=True)

# 其他

In [15]:
def set_device(row,col,write_voltage,tg,pulse_width):
    chip.set_op_mode2(read=False,from_row=True,clearv=False)
    chip.set_pulse_width(pulse_width)
    chip.write2(row_index=[row],col_index=[col],write_voltage=write_voltage,tg=tg)

def reset_device(row,col,write_voltage,tg,pulse_width):
    chip.set_op_mode2(read=False,from_row=False,clearv=False)
    chip.set_pulse_width(pulse_width)
    chip.write2(row_index=[row],col_index=[col],write_voltage=write_voltage,tg=tg)

def read_device(row,col,read_voltage,tg,gain):
    chip.set_op_mode2(read=True,from_row=True,clearv=False)
    chip.set_tia_gain(gain)
    vres = chip.read2(row_index=[row],col_index=[col],read_voltage=read_voltage,tg=tg,check_tia=True,sum=False)
    return chip.voltage_to_cond(vres[0,0])